# Clean Data

Basic cleaning steps for California housing data

Each row corresponds to a census area for 1990

In [73]:
import pandas as pd
import numpy as np

In [54]:
df = pd.read_csv('Data/CA_housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [55]:
print('Shape:', df.shape, '\n')
print(df.dtypes,'\n')
print('Has missing values:', df.isnull().any(), '\n')
print(df.describe())

Shape: (20640, 10) 

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object 

Has missing values: longitude             False
latitude              False
housing_median_age    False
total_rooms           False
total_bedrooms         True
population            False
households            False
median_income         False
median_house_value    False
ocean_proximity       False
dtype: bool 

          longitude      latitude  housing_median_age   total_rooms  \
count  20640.000000  20640.000000        20640.000000  20640.000000   
mean    -119.569704     35.631861           28.639486   2635.763081   
std        2.003532      2.135952           12.585558   2181.615252   
min     -124.350000     32.540000            1.000000      2.0000

Ok, looks like we need to address the missing values for total_bedrooms, change ocean proximity to one hot encoding, and potentially drop some outliers

Let's do the one hot encoding first

In [56]:
df.ocean_proximity.value_counts()

ocean_proximity
<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: count, dtype: int64

In [57]:
one_hot = pd.get_dummies(df['ocean_proximity'],dtype='float64', drop_first=True)
df = df.drop('ocean_proximity',axis = 1)

df = df.join(one_hot)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0.0,0.0,1.0,0.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0.0,0.0,1.0,0.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0.0,0.0,1.0,0.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0.0,0.0,1.0,0.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0.0,0.0,1.0,0.0


Now let's address the nulls

In [58]:
df[df.isnull().any(axis=1)]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
290,-122.16,37.77,47.0,1256.0,NaN,570.0,218.0,4.3750,161900.0,0.0,0.0,1.0,0.0
341,-122.17,37.75,38.0,992.0,NaN,732.0,259.0,1.6196,85100.0,0.0,0.0,1.0,0.0
538,-122.28,37.78,29.0,5154.0,NaN,3741.0,1273.0,2.5762,173400.0,0.0,0.0,1.0,0.0
563,-122.24,37.75,45.0,891.0,NaN,384.0,146.0,4.9489,247100.0,0.0,0.0,1.0,0.0
696,-122.10,37.69,41.0,746.0,NaN,387.0,161.0,3.9063,178400.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20267,-119.19,34.20,18.0,3620.0,NaN,3171.0,779.0,3.3409,220500.0,0.0,0.0,0.0,1.0
20268,-119.18,34.19,19.0,2393.0,NaN,1938.0,762.0,1.6953,167400.0,0.0,0.0,0.0,1.0
20372,-118.88,34.17,15.0,4260.0,NaN,1701.0,669.0,5.1033,410700.0,0.0,0.0,0.0,0.0
20460,-118.75,34.29,17.0,5512.0,NaN,2734.0,814.0,6.6073,258100.0,0.0,0.0,0.0,0.0


I will just drop these values since it is a very small percent of the data

In [59]:
df.dropna(inplace=True)

Now for outliers, which I will not remove but just want to get an idea of how many low population areas there are

In [64]:
df.population.quantile(0.02)

169.64

In [65]:
df.households.quantile(0.02)

58.0

In [66]:
df.query('households < 20')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
73,-122.29,37.81,46.0,12.0,4.0,18.0,7.0,0.4999,67500.0,0.0,0.0,1.0,0.0
1419,-122.05,37.97,16.0,60.0,10.0,65.0,19.0,6.1359,250000.0,0.0,0.0,1.0,0.0
1566,-121.96,37.74,2.0,200.0,20.0,25.0,9.0,15.0001,350000.0,0.0,0.0,0.0,0.0
1633,-122.22,37.88,20.0,95.0,13.0,31.0,15.0,2.4444,475000.0,0.0,0.0,1.0,0.0
1721,-122.39,38.00,33.0,44.0,6.0,23.0,11.0,4.1250,212500.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20181,-119.32,34.35,16.0,52.0,16.0,51.0,15.0,2.4750,225000.0,0.0,0.0,0.0,1.0
20248,-119.23,34.25,28.0,26.0,3.0,29.0,9.0,8.0000,275000.0,0.0,0.0,0.0,1.0
20352,-119.09,34.22,8.0,40.0,10.0,309.0,16.0,4.0208,52500.0,0.0,0.0,0.0,1.0
20353,-119.05,34.13,12.0,57.0,22.0,69.0,15.0,5.0066,275000.0,0.0,0.0,0.0,1.0


In [37]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
count,20200.000000,20200.000000,20200.000000,20200.000000,20200.000000,20200.000000,20200.000000,20200.000000,20200.000000,20200.000000,20200.000000,20200.000000,20200.000000
mean,-119.568155,35.626089,28.630495,2664.480149,543.538317,1440.765198,504.945495,3.870819,207033.586287,0.315396,0.000248,0.110891,0.128911
std,2.001672,2.134220,12.554341,2181.784200,420.373963,1130.054382,381.014733,1.881940,115276.261083,0.464685,0.015731,0.314005,0.335110
min,-124.350000,32.540000,1.000000,19.000000,5.000000,100.000000,6.000000,0.499900,14999.000000,0.000000,0.000000,0.000000,0.000000
25%,-121.790000,33.930000,18.000000,1471.000000,300.000000,800.000000,284.000000,2.568050,120000.000000,0.000000,0.000000,0.000000,0.000000
50%,-118.490000,34.250000,29.000000,2145.000000,438.000000,1175.000000,412.000000,3.538950,180000.000000,0.000000,0.000000,0.000000,0.000000
75%,-118.010000,37.710000,37.000000,3161.000000,650.250000,1732.000000,607.000000,4.744075,264900.000000,1.000000,0.000000,0.000000,0.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000,1.000000,1.000000,1.000000,1.000000


Now standardizing the data, for now I will be standardizing the one hot encoding, but will explore the difference in doing that vs not

In [82]:
df = (df-df.mean())/df.std()

df.to_pickle('Data/CA_housing.pkl')